# Box Plots

In [1]:
# Down the file to local
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/QR9YeprUYhOoLafzlLspAw/survey-results-public.sqlite

--2025-02-10 20:44:53--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/QR9YeprUYhOoLafzlLspAw/survey-results-public.sqlite
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 211415040 (202M) [application/octet-stream]
Saving to: ‘survey-results-public.sqlite’

survey-results-publ 100%[===================>] 201.62M  2.36MB/s    in 1m 57s  

2025-02-10 20:46:51 (1.73 MB/s) - ‘survey-results-public.sqlite’ saved [211415040/211415040]



In [2]:
# Import all package
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Connect the Database

In [3]:
conn = sqlite3.connect('survey-results-public.sqlite')

In [4]:
#  Read The database
QUERY = """
SELECT COUNT(*) FROM main
"""
df = pd.read_sql_query(QUERY, conn)
df

,COUNT(*)
0,65437


In [5]:
# List of All Tables
QUERY ="""
SELECT name as Table_Name
FROM sqlite_master
WHERE type='table'
"""
table_info = pd.read_sql_query(QUERY, conn)
table_info

,Table_Name
0,main
